In [51]:
# Used libraries 
import pandas as pd
import os
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import spacy
import re
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer



In [52]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mervetaskaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mervetaskaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mervetaskaya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mervetaskaya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [53]:
def read_data():
    df_artists = pd.read_csv('data/spotify_artists.csv', sep=',') 
    df_tracks = pd.read_csv('data/spotify_tracks.csv', sep=',')   
    return df_artists, df_tracks 
     
df_artists, df_tracks = read_data()

In [54]:
pd.set_option('display.max_columns', None)
df_artists.head(5)  

,Unnamed: 0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type
0,0,44,23230,"['sertanejo', 'sertanejo pop', 'sertanejo trad...",4mGnpjhqgx4RUdsIJiURdo,Juliano Cezar,0wmDmAILuW9e2aRttkl4aC,track_9,artist
1,1,22,313,[],1dLnVku4VQUOLswwDFvRc9,The Grenadines,4wqwj0gA8qPZKLl5WVqXml,track_30,artist
2,2,26,1596,['danish pop rock'],6YVY310fjfUzKi8hiqR7iK,Gangway,1bFqWDbvHmZe2f4Nf9qaD8,track_38,artist
3,3,31,149,['uk alternative pop'],2VElyouiCfoYPDJluzwJwK,FADES,3MFSUBAidPzRBbIS7BDj1S,track_34,artist
4,4,21,11,['french baroque'],4agVy03qW8juSysCTUOuDI,Jean-Pierre Guignon,2r3q57FhxdsCyYr0kuDq4b,track_26,artist


In [55]:
#check in df_artists if we have duplicates via id
df_artists[df_artists.duplicated(['id'])]

,Unnamed: 0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type


In [56]:
#check if there are missing genres
df_artists['genres'].value_counts()

#remove the rows with missing genres
df_artists = df_artists[df_artists['genres'] != '[]']
df_artists.head(5)

,Unnamed: 0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type
0,0,44,23230,"['sertanejo', 'sertanejo pop', 'sertanejo trad...",4mGnpjhqgx4RUdsIJiURdo,Juliano Cezar,0wmDmAILuW9e2aRttkl4aC,track_9,artist
2,2,26,1596,['danish pop rock'],6YVY310fjfUzKi8hiqR7iK,Gangway,1bFqWDbvHmZe2f4Nf9qaD8,track_38,artist
3,3,31,149,['uk alternative pop'],2VElyouiCfoYPDJluzwJwK,FADES,3MFSUBAidPzRBbIS7BDj1S,track_34,artist
4,4,21,11,['french baroque'],4agVy03qW8juSysCTUOuDI,Jean-Pierre Guignon,2r3q57FhxdsCyYr0kuDq4b,track_26,artist
9,9,25,992,['classic finnish pop'],5ijoPCUhV0dW8EJ7aPGvXK,Pepe Willberg & The Paradise,6WVlVPoesX2zVvJzZJUj6a,track_8,artist


In [57]:
#convert the string to list
import ast
df_artists['genres'] = df_artists['genres'].apply(ast.literal_eval)
df_artists.head(5)    

,Unnamed: 0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type
0,0,44,23230,"[sertanejo, sertanejo pop, sertanejo tradicion...",4mGnpjhqgx4RUdsIJiURdo,Juliano Cezar,0wmDmAILuW9e2aRttkl4aC,track_9,artist
2,2,26,1596,[danish pop rock],6YVY310fjfUzKi8hiqR7iK,Gangway,1bFqWDbvHmZe2f4Nf9qaD8,track_38,artist
3,3,31,149,[uk alternative pop],2VElyouiCfoYPDJluzwJwK,FADES,3MFSUBAidPzRBbIS7BDj1S,track_34,artist
4,4,21,11,[french baroque],4agVy03qW8juSysCTUOuDI,Jean-Pierre Guignon,2r3q57FhxdsCyYr0kuDq4b,track_26,artist
9,9,25,992,[classic finnish pop],5ijoPCUhV0dW8EJ7aPGvXK,Pepe Willberg & The Paradise,6WVlVPoesX2zVvJzZJUj6a,track_8,artist


### Preprocessing | Text standardization or normalization

In [58]:
#put everything in lower case
df_artists['genres'] = df_artists['genres'].apply(lambda x: [genre.strip().lower() for genre in x])

In [59]:
#remove the characters that are not letters (both uppercase and lowercase) or whitespace
df_artists['cleaned_genres'] = df_artists['genres'].apply(lambda x: [re.sub(r'[^a-zA-Z\s]', '', genre) for genre in x])
df_artists.head(5)

,Unnamed: 0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type,cleaned_genres
0,0,44,23230,"[sertanejo, sertanejo pop, sertanejo tradicion...",4mGnpjhqgx4RUdsIJiURdo,Juliano Cezar,0wmDmAILuW9e2aRttkl4aC,track_9,artist,"[sertanejo, sertanejo pop, sertanejo tradicion..."
2,2,26,1596,[danish pop rock],6YVY310fjfUzKi8hiqR7iK,Gangway,1bFqWDbvHmZe2f4Nf9qaD8,track_38,artist,[danish pop rock]
3,3,31,149,[uk alternative pop],2VElyouiCfoYPDJluzwJwK,FADES,3MFSUBAidPzRBbIS7BDj1S,track_34,artist,[uk alternative pop]
4,4,21,11,[french baroque],4agVy03qW8juSysCTUOuDI,Jean-Pierre Guignon,2r3q57FhxdsCyYr0kuDq4b,track_26,artist,[french baroque]
9,9,25,992,[classic finnish pop],5ijoPCUhV0dW8EJ7aPGvXK,Pepe Willberg & The Paradise,6WVlVPoesX2zVvJzZJUj6a,track_8,artist,[classic finnish pop]


### Stoppwords

In [60]:
# extract all genres from the cleaned_genres column
all_genres = [genre for sublist in df_artists['cleaned_genres'] for genre in sublist]
genres_stopwords = [word for word in all_genres if word.lower() in stop_words]

# count of stopwords in genres data
stopword_counts = Counter(genres_stopwords)
stopword_counts

Counter()

### Tokenization

In [61]:
df_artists['tokenized_genre'] = df_artists['cleaned_genres'].apply(lambda x: [nltk.word_tokenize(genre) for genre in x])
df_artists.head(5)

,Unnamed: 0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type,cleaned_genres,tokenized_genre
0,0,44,23230,"[sertanejo, sertanejo pop, sertanejo tradicion...",4mGnpjhqgx4RUdsIJiURdo,Juliano Cezar,0wmDmAILuW9e2aRttkl4aC,track_9,artist,"[sertanejo, sertanejo pop, sertanejo tradicion...","[[sertanejo], [sertanejo, pop], [sertanejo, tr..."
2,2,26,1596,[danish pop rock],6YVY310fjfUzKi8hiqR7iK,Gangway,1bFqWDbvHmZe2f4Nf9qaD8,track_38,artist,[danish pop rock],"[[danish, pop, rock]]"
3,3,31,149,[uk alternative pop],2VElyouiCfoYPDJluzwJwK,FADES,3MFSUBAidPzRBbIS7BDj1S,track_34,artist,[uk alternative pop],"[[uk, alternative, pop]]"
4,4,21,11,[french baroque],4agVy03qW8juSysCTUOuDI,Jean-Pierre Guignon,2r3q57FhxdsCyYr0kuDq4b,track_26,artist,[french baroque],"[[french, baroque]]"
9,9,25,992,[classic finnish pop],5ijoPCUhV0dW8EJ7aPGvXK,Pepe Willberg & The Paradise,6WVlVPoesX2zVvJzZJUj6a,track_8,artist,[classic finnish pop],"[[classic, finnish, pop]]"


In [62]:
stemmer = PorterStemmer()
df_artists['stemmed_genres'] = df_artists['cleaned_genres'].apply(
    lambda x: [stemmer.stem(genre) for genre in x]
)

In [63]:
lemmatizer = WordNetLemmatizer()
df_artists['lemmatized_genres'] = df_artists['cleaned_genres'].apply(
    lambda x: [lemmatizer.lemmatize(genre) for genre in x]
)


In [64]:
df_artists['pos_tags_genres'] = df_artists['cleaned_genres'].apply(
    lambda x: [nltk.pos_tag([genre]) for genre in x]
)

In [65]:
#df_artists['cleaned_genres', 'stemmed_genres', 'lemmatized_genres', 'pos_tags_genres'].head()

#show just cleand genres and stemmed genres
df_artists[['cleaned_genres', 'stemmed_genres', 'lemmatized_genres', 'pos_tags_genres', 'tokenized_genre']].head()


,cleaned_genres,stemmed_genres,lemmatized_genres,pos_tags_genres,tokenized_genre
0,"[sertanejo, sertanejo pop, sertanejo tradicion...","[sertanejo, sertanejo pop, sertanejo tradicion...","[sertanejo, sertanejo pop, sertanejo tradicion...","[[(sertanejo, NN)], [(sertanejo pop, NN)], [(s...","[[sertanejo], [sertanejo, pop], [sertanejo, tr..."
2,[danish pop rock],[danish pop rock],[danish pop rock],"[[(danish pop rock, NN)]]","[[danish, pop, rock]]"
3,[uk alternative pop],[uk alternative pop],[uk alternative pop],"[[(uk alternative pop, NN)]]","[[uk, alternative, pop]]"
4,[french baroque],[french baroqu],[french baroque],"[[(french baroque, NN)]]","[[french, baroque]]"
9,[classic finnish pop],[classic finnish pop],[classic finnish pop],"[[(classic finnish pop, NN)]]","[[classic, finnish, pop]]"
